In [1]:
# Import numpy and pandas
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

In [7]:
# Read and preview data
trans_data_file = rf'data\QVI_transaction_data.xlsx'
cust_data_file = rf'data\QVI_purchase_behaviour.csv'

trans_data = pd.read_excel(trans_data_file)
cust_data = pd.read_csv(cust_data_file)

In [9]:
print('Transaction data preview:\n', trans_data.head(), "\n\n")
print("Customer data preview:\n", cust_data.head())

Transaction data preview:
     DATE  STORE_NBR  LYLTY_CARD_NBR  TXN_ID  PROD_NBR  \
0  43390          1            1000       1         5   
1  43599          1            1307     348        66   
2  43605          1            1343     383        61   
3  43329          2            2373     974        69   
4  43330          2            2426    1038       108   

                                  PROD_NAME  PROD_QTY  TOT_SALES  
0    Natural Chip        Compny SeaSalt175g         2        6.0  
1                  CCs Nacho Cheese    175g         3        6.3  
2    Smiths Crinkle Cut  Chips Chicken 170g         2        2.9  
3    Smiths Chip Thinly  S/Cream&Onion 175g         5       15.0  
4  Kettle Tortilla ChpsHny&Jlpno Chili 150g         3       13.8   


Customer data preview:
    LYLTY_CARD_NBR               LIFESTAGE PREMIUM_CUSTOMER
0            1000   YOUNG SINGLES/COUPLES          Premium
1            1002   YOUNG SINGLES/COUPLES       Mainstream
2            1003       

In [13]:
# Check for and view duplicates
print("Transaction data duplicates:\n", trans_data[trans_data.duplicated(keep=False)], "\n")
print("Customer data duplicates:\n", cust_data[cust_data.duplicated(keep=False)], "\n")

# Drop duplicates
trans_data = trans_data.drop_duplicates()
cust_data = cust_data.drop_duplicates()

# Check for nulls. In this case there are none.
print("Transaction data Nulls:\n", trans_data.isnull().any(), "\n")
print("Customer data Nulls:\n", cust_data.isnull().any())

Transaction data duplicates:
 Empty DataFrame
Columns: [DATE, STORE_NBR, LYLTY_CARD_NBR, TXN_ID, PROD_NBR, PROD_NAME, PROD_QTY, TOT_SALES]
Index: [] 

Customer data duplicates:
 Empty DataFrame
Columns: [LYLTY_CARD_NBR, LIFESTAGE, PREMIUM_CUSTOMER]
Index: [] 

Transaction data Nulls:
 DATE              False
STORE_NBR         False
LYLTY_CARD_NBR    False
TXN_ID            False
PROD_NBR          False
PROD_NAME         False
PROD_QTY          False
TOT_SALES         False
dtype: bool 

Customer data Nulls:
 LYLTY_CARD_NBR      False
LIFESTAGE           False
PREMIUM_CUSTOMER    False
dtype: bool


In [14]:
# Extract weight data 
extr = trans_data['PROD_NAME'].str.extract(r'(\d{1,4}g)', expand=False, flags=re.IGNORECASE)

# Remove g from weight
extr = extr.str.extract(r'(\d{1,4})')

# check to see if any row did not have a matching weight string
print("\nIs any extraction null?\n", extr.isnull().any().values)

# add extracted weight to dataframe
trans_data['WEIGHT'] = extr.astype(np.float32)

# also update the 'DATE' column from Excel's numeric format to a proper date time format.
trans_data['DATE'] = pd.to_datetime(trans_data['DATE'], unit='D', origin=pd.Timestamp('1899-12-30'))

# Preview
print("\nPreview updated dataframe\n", trans_data.head())


Is any extraction null?
 [False]

Preview updated dataframe
         DATE  STORE_NBR  LYLTY_CARD_NBR  TXN_ID  PROD_NBR  \
0 2018-10-17          1            1000       1         5   
1 2019-05-14          1            1307     348        66   
2 2019-05-20          1            1343     383        61   
3 2018-08-17          2            2373     974        69   
4 2018-08-18          2            2426    1038       108   

                                  PROD_NAME  PROD_QTY  TOT_SALES  WEIGHT  
0    Natural Chip        Compny SeaSalt175g         2        6.0   175.0  
1                  CCs Nacho Cheese    175g         3        6.3   175.0  
2    Smiths Crinkle Cut  Chips Chicken 170g         2        2.9   170.0  
3    Smiths Chip Thinly  S/Cream&Onion 175g         5       15.0   175.0  
4  Kettle Tortilla ChpsHny&Jlpno Chili 150g         3       13.8   150.0  


In [43]:
# For transaction data, the most interesting columns
trans_sale_columns = ["TOT_SALES"]

trans_cust_data = trans_data.join(cust_data, on="LYLTY_CARD_NBR", how="left", lsuffix="t", rsuffix="c")

# trans_cust_data.groupby("LIFESTAGE", group_keys=False)[trans_sale_columns].agg(['min', 'mean', 'max'])
# trans_cust_data.groupby("PREMIUM_CUSTOMER", group_keys=False)[trans_sale_columns].agg(['min', 'mean', 'max', 'count'])
trans_cust_data.groupby(["PREMIUM_CUSTOMER", "LIFESTAGE"], group_keys=False)[trans_sale_columns].agg(['mean', 'max', 'count', 'sum'])['TOT_SALES'].sort_values('sum')

mean   max  count       sum
PREMIUM_CUSTOMER LIFESTAGE                                              
Premium          NEW FAMILIES            7.350000  18.4    574   4218.90
Mainstream       NEW FAMILIES            7.418228  22.0    779   5778.80
Budget           NEW FAMILIES            7.157226  13.0    941   6734.95
                 MIDAGE SINGLES/COUPLES  7.370277  29.5   1514  11158.60
Premium          YOUNG SINGLES/COUPLES   7.434846  23.0   2270  16877.10
                 MIDAGE SINGLES/COUPLES  7.357186  22.0   2331  17149.60
                 OLDER FAMILIES          7.402636  16.2   2352  17411.00
                 YOUNG FAMILIES          7.362086  17.6   2536  18670.25
Mainstream       YOUNG FAMILIES          7.333683  27.0   2816  20651.65
                 OLDER FAMILIES          7.404958  27.0   2834  20985.65
                 MIDAGE SINGLES/COUPLES  7.253399  22.8   3060  22195.40
Budget           YOUNG SINGLES/COUPLES   7.237264  25.5   3502  25344.90
Premium          RETIREES                7.365276  29.5   3810  28061.70
Budget           YOUNG FAMILIES          7.407366  28.5   4039  29918.35
                 RETIREES                7.362924  21.6   4333  31903.55
Mainstream       OLDER SINGLES/COUPLES   7.299401  27.0   4842  35343.70
Budget           OLDER FAMILIES          7.412979  29.5   4800  35582.30
Premium          OLDER SINGLES/COUPLES   7.375357  25.5   4837  35674.60
Budget           OLDER SINGLES/COUPLES   7.243128  23.6   4991  36150.45
Mainstream       RETIREES                7.346375  27.0   6180  45400.60
                 YOUNG SINGLES/COUPLES   7.366247  27.0   7432  54745.95